In [98]:
import pickle
import os
import numpy as np
import glob

In [114]:
dataset = '<dataset_name>'
exp_name = '<exp_name>'


root = './results/'
experiment = f'{dataset}_tuning_{exp_name}'
prefix = f'{dataset}-'
num_configurations = 512

In [119]:
# parse aggregated results

def parse_results(stream):
    conf = dict()
    for line in stream:
        line = line.strip().split(':')
        if line[0].strip() == 'Accuracy':
            result = line[1].strip().split('±')
            mean = float(result[0].strip())
            std = float(result[1].strip())
        elif line[0].strip()[:2] == '--' and line[0].strip()[2] != '-':
            key = line[0].strip()[2:]
            value = line[1].strip()
            conf[key] = value
    return (mean, std, conf)

folder = os.path.join(root, experiment, prefix+'{}')
res_list = list()
for i in range(num_configurations):
    path = os.path.join(folder.format(i), 'result.txt')
    try:
        with open(path, 'r') as handle:
            res_list.append((*parse_results(handle), i))
    except FileNotFoundError:
        continue
print('Found {} result(s).'.format(len(res_list)))

Found 64 result(s).


In [120]:
# rank and inspect

ranked = sorted(res_list, key=lambda x: -x[0])
inspect_args = [
    'batch_size',
    'drop_rate',
    'final_readout',
    'lr',
    'lr_scheduler_decay_rate',
    'lr_scheduler_decay_steps']
k = 5

for i in range(k):
    print('-------------------------------------------------------')
    print('{} ({}): {:.4f} ± {:.4f}\n'.format(i+1, ranked[i][3], ranked[i][0], ranked[i][1]))
    for key in ranked[i][2]:
        if key not in inspect_args:
            continue
        print('\t{}: {}'.format(key, ranked[i][2][key]))
    print()

-------------------------------------------------------
1 (11): 0.5731 ± 0.0156

	batch_size: 32
	drop_rate: 0.0
	final_readout: sum
	lr: 0.001
	lr_scheduler_decay_rate: 0.9
	lr_scheduler_decay_steps: 20

-------------------------------------------------------
2 (10): 0.5689 ± 0.0125

	batch_size: 32
	drop_rate: 0.0
	final_readout: sum
	lr: 0.001
	lr_scheduler_decay_rate: 0.9
	lr_scheduler_decay_steps: 50

-------------------------------------------------------
3 (32): 0.5683 ± 0.0134

	batch_size: 128
	drop_rate: 0.0
	final_readout: mean
	lr: 0.001
	lr_scheduler_decay_rate: 0.5
	lr_scheduler_decay_steps: 50

-------------------------------------------------------
4 (8): 0.5681 ± 0.0181

	batch_size: 32
	drop_rate: 0.0
	final_readout: sum
	lr: 0.001
	lr_scheduler_decay_rate: 0.5
	lr_scheduler_decay_steps: 50

-------------------------------------------------------
5 (9): 0.5679 ± 0.0250

	batch_size: 32
	drop_rate: 0.0
	final_readout: sum
	lr: 0.001
	lr_scheduler_decay_rate: 0.5
	lr_sc